In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Bidirectional, Attention, GlobalAveragePooling1D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
tf.get_logger().setLevel('ERROR')

In [2]:
data = pd.read_csv('data.csv', encoding='utf-8', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])
data.drop(columns=['ids', 'date', 'flag', 'user'], inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [3]:
data = shuffle(data, random_state=42).reset_index(drop=True)
data

,target,text
0,0,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,0,"@misstoriblack cool , i have no tweet apps fo..."
2,0,@TiannaChaos i know just family drama. its la...
3,0,School email won't open and I have geography ...
4,0,upper airways problem
...,...,...
1599995,0,this song's middle change just doesn't want to...
1599996,4,@officialnjonas Good luck with that
1599997,0,@ProudGamerTweet I rather average 32370
1599998,0,Pickin up @misstinayao waitin on @sadittysash ...


In [4]:
df =data

In [5]:
# Convert 'target' to bnary sentiment labels (0 or 1)
df['target'] = df['target'].apply(lambda x: 0 if x == 0 else 1)

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert data to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_df['text'].values, train_df['target'].values))
test_data = tf.data.Dataset.from_tensor_slices((test_df['text'].values, test_df['target'].values))


In [34]:
# Original shape
original_shape = df.shape
print("Original DataFrame shape:", original_shape)

# Calculate the new size (1/100 of the original)
new_size = original_shape[0] // 100 # integer division to get the whole number
print("New size for training data:", new_size)

# Randomly sample the training data
smaller_train_df = train_df.sample(n=new_size, random_state=42)

# Check the shape of the new training dataset
print("Smaller Training DataFrame shape:", smaller_train_df.shape)

smaller_train_data = tf.data.Dataset.from_tensor_slices((smaller_train_df['text'].values, smaller_train_df['target'].values))

smaller_test_df = test_df.sample(n=new_size, random_state=42)

# Check the shape of the new training dataset
print("Smaller Training DataFrame shape:", smaller_test_df.shape)

smaller_test_data = tf.data.Dataset.from_tensor_slices((smaller_test_df['text'].values, smaller_test_df['target'].values))

Original DataFrame shape: (1600000, 2)
New size for training data: 16000
Smaller Training DataFrame shape: (16000, 2)
Smaller Training DataFrame shape: (16000, 2)


In [35]:
import tensorflow_text
bert_model_url = "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2"  # SST-2 model trained for sentiment analysis
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

bert_preprocessor = hub.KerasLayer(preprocessor_url)
bert_encoder = hub.KerasLayer(bert_model_url)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Bidirectional, Attention, GlobalAveragePooling1D, Dropout, Dense
from tensorflow.keras.optimizers import Adam

def build_model():
    # Input layer for text
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

    # BERT preprocessor and encoder (assuming you have a pre-trained BERT model)
    preprocessed_text = bert_preprocessor(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Extract the sequence output from the BERT encoder (not just pooled output)
    bert_sequence_output = outputs['sequence_output']
    
    # Add a Bidirectional LSTM Layer
    lstm_output = Bidirectional(LSTM(units=128, return_sequences=True))(bert_sequence_output)
    
    # Attention Layer for sequence processing
    attention_output = Attention()([lstm_output, lstm_output])
    
    # Apply Global Average Pooling to the attention output
    pooled_output = GlobalAveragePooling1D()(attention_output)
    
    # Dropout for regularization
    net = Dropout(rate=0.3)(pooled_output)  # Set dropout rate as 0.3

    # Add Dense Layers
    net = Dense(128, activation='relu')(net)  # First dense layer
    net = Dropout(rate=0.4)(net)  # Dropout for regularization
    net = Dense(64, activation='tanh')(net)  # First dense layer
    net = Dropout(rate=0.4)(net)  # Dropout for regularization
    net = Dense(32, activation='relu')(net)  # Second dense layer
    net = Dropout(rate=0.4)(net)
    
    net = Dense(16, activation='relu')(net)  # Third dense layer
    net = Dropout(rate=0.4)(net)
    
    # Final output layer with sigmoid activation for binary classification
    net = Dense(1, activation='sigmoid', name='classifier')(net)
    
    # Build the model
    model = tf.keras.Model(inputs=text_input, outputs=net)

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=1e-4),  # Set learning rate to 1e-4
        loss='binary_crossentropy',  # Binary classification
        metrics=['accuracy']
    )

    return model

# Build the model
model = build_model()

# Display model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [25]:

# Assuming X_train and y_train are your training data
# Train the model
model.fit(smaller_train_df['text'], smaller_train_df['target'], epochs=2, batch_size=32)  # Use a reasonable batch size


Epoch 1/2
500/500 [==============================] - 1020s 2s/step - loss: 0.5821 - accuracy: 0.6855
Epoch 2/2
500/500 [==============================] - 1066s 2s/step - loss: 0.5181 - accuracy: 0.7546


In [36]:
def build_model(hp):
    # Input layer for text
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

    # BERT preprocessor and encoder (assuming you have a pre-trained BERT model)
    preprocessed_text = bert_preprocessor(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Extract the sequence output from the BERT encoder (not just pooled output)
    bert_sequence_output = outputs['sequence_output']
    
    # Add a Bidirectional LSTM Layer
    lstm_output = Bidirectional(LSTM(
        units=hp.Int('lstm_units', min_value=20, max_value=50, step=5),
        return_sequences=True))(bert_sequence_output)
    
    # Attention Layer for sequence processing
    attention_output = Attention()([lstm_output, lstm_output])
    
    # Apply Global Average Pooling to the attention output
    pooled_output = GlobalAveragePooling1D()(attention_output)
    
    # Dropout for regularization
    net = Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.7, step=0.1))(pooled_output)

    # Add Dense Layers
    net = Dense(
        units=hp.Int('dense_units', min_value=36, max_value=120, step=16),
        activation='relu')(net)
    net = Dropout(rate=hp.Float('dropout_rate_2', min_value=0.2, max_value=0.7, step=0.1))(net)
    net = Dense(
        units=hp.Int('dense_units', min_value=12, max_value=30, step=4),
        activation='relu')(net)
    net = Dropout(rate=hp.Float('dropout_rate_2', min_value=0.2, max_value=0.7, step=0.1))(net)
    net = Dense(1, activation='sigmoid', name='classifier')(net)  # Final output layer for binary classification
    
    # Build the model
    model = tf.keras.Model(inputs=text_input, outputs=net)

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',  # Binary classification
        metrics=['accuracy']
    )

    return model


tuner = kt.BayesianOptimization(
    build_model,
    objective='accuracy',
    max_trials=10,  # Number of trials to run
    executions_per_trial=2,  # Run each trial once
    directory='BERT_LSTM_new',  # Save results here
    project_name='bert_lstm_attention_tuning'
)

# Perform the hyperparameter search
tuner.search(smaller_train_df['text'], smaller_train_df['target'], epochs=2, batch_size=32)  # Use a reasonable batch size


# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hp.values}")


Trial 10 Complete [01h 34m 32s]
accuracy: 0.6755937337875366

Best accuracy So Far: 0.7838749885559082
Total elapsed time: 15h 44m 29s
Best Hyperparameters: {'lstm_units': 40, 'dropout_rate': 0.5, 'dense_units': 52, 'dropout_rate_2': 0.30000000000000004, 'learning_rate': 0.00027081735801911806}


In [37]:
model = build_model(best_hp)
model.fit(smaller_train_df['text'], smaller_train_df['target'], epochs=50, batch_size=64)


Epoch 1/50
250/250 [==============================] - 1228s 5s/step - loss: 0.5249 - accuracy: 0.7478
Epoch 2/50
250/250 [==============================] - 1271s 5s/step - loss: 0.4744 - accuracy: 0.7791
Epoch 3/50
250/250 [==============================] - 1277s 5s/step - loss: 0.4624 - accuracy: 0.7907
Epoch 4/50
250/250 [==============================] - 1247s 5s/step - loss: 0.4489 - accuracy: 0.7928
Epoch 5/50
250/250 [==============================] - 1249s 5s/step - loss: 0.4389 - accuracy: 0.8020
Epoch 6/50
250/250 [==============================] - 1253s 5s/step - loss: 0.4309 - accuracy: 0.8059
Epoch 7/50
250/250 [==============================] - 1248s 5s/step - loss: 0.4217 - accuracy: 0.8104
Epoch 8/50
250/250 [==============================] - 1250s 5s/step - loss: 0.4179 - accuracy: 0.8122
Epoch 9/50
250/250 [==============================] - 1243s 5s/step - loss: 0.4110 - accuracy: 0.8173
Epoch 10/50
250/250 [==============================] - 1247s 5s/step - loss: 0.400

In [ ]:
y_pred_prob = model.predict(test_df['text'])
y_pred = (y_pred_prob > 0.5).astype(int)
y_true = test_df['target']


   14/10000 [..............................] - ETA: 8:28:41

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

In [ ]:
# Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Precision
precision = precision_score(y_true, y_pred)

# Recall
recall = recall_score(y_true, y_pred)

# F1 Score
f1 = f1_score(y_true, y_pred)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.7937
Precision: 0.7955
Recall: 0.7916
F1 Score: 0.7936
Confusion Matrix:
[[6354 1631]
 [1670 6345]]


In [41]:
model.predict(["I am happy"])

1/1 [==============================] - 1s 938ms/step


array([[0.9999977]], dtype=float32)

In [42]:
model.save('LSTM_BERT/model')